In [ ]:
import MLJ
import DataFrames as DF
import CSV
using DataFramesMeta
using StatsPlots
using LaTeXStrings

In [ ]:
include("PreprocessAerosolData.jl")
include("ActivationEmulatorModels.jl")

In [ ]:
function plot_single_var_profile(dataset_filename::String, x_axis_var::Symbol, mach_filenames, mach_labels; kwargs...)
    X_test, Y_test = read_aerosol_dataset(dataset_filename)
    ARG_S_max = exp.(preprocess_aerosol_data(X_test).log_ARG_S_max)
    PySDM_S_max = exp.(Y_test)
    x_range = X_test[!, x_axis_var]
    p = plot(x_range, PySDM_S_max, label="PySDM", xlabel=x_axis_var, ylabel="S_max", yscale=:log10; kwargs...)
    plot!(x_range, ARG_S_max, label="ARG")
    for (mach_filename, mach_label) in zip(mach_filenames, mach_labels)
        mach = MLJ.machine(mach_filename)
        pred_S_max = exp.(MLJ.predict(mach, X_test))
        plot!(x_range, pred_S_max, label=mach_label)
    end
    return p
end

In [ ]:
mach_filenames = ["emulators/1modal_evotree_machine1.jls", "emulators/1modal_nn_machine1.jls"]
mach_labels = ["EvoTree model", "NN model"]
p1 = plot_single_var_profile("datasets/single_var_profiles/Silva_Ma_Fig7_mode_N.csv", :mode_1_N, mach_filenames, mach_labels, xscale=:log10)
p2 = plot_single_var_profile("datasets/single_var_profiles/Silva_Ma_Fig7_mode_mean.csv", :mode_1_mean, mach_filenames, mach_labels, xscale=:log10)
p3 = plot_single_var_profile("datasets/single_var_profiles/Silva_Ma_Fig7_velocity.csv", :velocity, mach_filenames, mach_labels, xscale=:log10)
p4 = plot_single_var_profile("datasets/single_var_profiles/Silva_Ma_Fig7_kappa.csv", :mode_1_kappa, mach_filenames, mach_labels)
plot(p1, p2, p3, p4)